1. sudo tar -C /usr -xzf ollama-linux-amd64.tgz
2. ollama serve
Open a new terminal
#3. ollama run llama3.3
#4. ollama run mistral


In [1]:
from langchain_community.llms.ollama import Ollama
import re
from langchain.vectorstores import FAISS
import pandas as pd
from build_database import get_embedding_function
import gc



In [ ]:
#test
llama_model = Ollama(model="llama3.3")
s= llama_model('tell me a joke')
s

In [3]:
# Hyperparameter Tuning Wrapper
class ModelHyperparameterTuning:
    def __init__(self, chunk_size, method):
        self.chunk_size = chunk_size
        self.method = method

    def load_vignettes(self, filepath, cat):
        """Load vignettes, categories, and ground truth labels from the CSV file."""
        data = pd.read_csv(filepath)
        stress_vignettes = data#[data["Category"] == cat]
        vignettes = {}
        for _, row in stress_vignettes.iterrows():
            vignette_id = row['Vignette ID']
            category = row['Category']
            vignette = f"{row['Referral']}\n{row['Presenting Symptoms']}\n{row['Additional Background Information']}"
            vignettes[vignette_id] = {
                "category": category,
                "vignette_text": vignette.strip(),
                "label": row['Label']
            }
        return vignettes

    def generate_diagnosis(self, rag_text, vignette_text, cot_text):
        prompt = f"""
        You are a professional clinician tasked with diagnosing based on the following vignette. 


        Clinical Guidelines:
        {rag_text}

        Vignette:
        {vignette_text}
        
        Chain of Thought:
        {cot_text}
        
        
        Consider clinical guidelines and chian of thought carefully to reach a thoughtful diagnosis based on the vignette's details.

        If you don't know the answer, just say that you don't know; don't try to make up an answer.
        """
        response = llama_model.invoke(prompt)
        return response

    def rag(self, vignette):
        embedding_function = get_embedding_function()
        db = FAISS.load_local("Chroma_path", embedding_function, allow_dangerous_deserialization=True)
        results = db.similarity_search(vignette, k=self.chunk_size)
        context_text = "\n\n---\n\n".join([doc.page_content for doc in results])
        return context_text

    def rag_compress(self, vignette):
        context_text = self.rag(vignette)
        prompt = (
            "You are a professional summarizer and content organizer. Given the following extracted "
            "information, compress and restructure it into a concise, well-organized summary while "
            "preserving all critical details. Ensure the language is clear and the structure is logical.\n\n"
            "Extracted Chunks:\n"
            f"{context_text}\n\n"
            "Compressed and Restructured Summary:"
        )
        compress_response = llama_model.invoke(prompt)
        return compress_response

    def cot(self): #cot_all
        with open("cot_all.txt", "r", encoding="utf-8") as file:
            cot_text = file.read()
        return cot_text

    def cot_compress(self):
        cot_text = self.cot()
        prompt = (
            "You are a professional summarizer and content organizer. Given the following extracted "
            "information, compress and restructure it into a concise, well-organized summary while "
            "preserving all critical details. Ensure the language is clear and the structure is logical.\n\n"
            "Extracted Chunks:\n"
            f"{cot_text}\n\n"
            "Compressed and Restructured Summary:"
        )
        compress_response = llama_model.invoke(prompt)
        return compress_response

    def get_rag_text(self, vignette_text):
        if "rag" in self.method:
            return self.rag_compress(vignette_text) if "rag compress" in self.method else self.rag(vignette_text)
        return ""

    def get_cot_text(self):
        if "cot" in self.method:
            return self.cot_compress() if "cot compress" in self.method else self.cot()
        return ""

    def extract_diagnosis(self, vignette_text):
        """Use the model to extract the diagnosis from the vignette text."""
        prompt = f"Extract the final diagnosis from this text:\n\n{vignette_text}"
        response = llama_model(prompt)
        return response.strip()

    def compare_diagnosis(self, model_diagnosis, ground_truth_label):
        """Use the model to determine if the extracted diagnosis matches the ground truth."""
        prompt = (
            f"Given the diagnosis: '{model_diagnosis}'\n\n"
            f"The ground truth diagnosis is: '{ground_truth_label}'.\n\n"
            f"Does the final diagnosis match the ground truth? Give the similarity score (0-100) and provide reasoning."
            f"Example: 'Score: 85. The diagnoses are very similar because...")
        response = mistral_model(prompt).strip()
        #prompt_score = f' {response},focus on the similairty score,give the similarity score, only numers without other string'
        #score_extract = mistral_model(response)
        #score_match = re.search(r"(\d+)", score_extract)
        #print(score_extract)

        score_match = re.search(r"(\d+)", response)

        if score_match:
            score = int(score_match.group(1))
        else:
            score = -1
            #raise ValueError("Unable to parse similarity score from the response.")
        return score


    def evaluate_model(self, vignettes):
        """Evaluate model performance by comparing predictions with ground truth labels."""
        correct_counts = {"Mood": 0, "Anxiety": 0, "Stress": 0}
        total_counts = {"Mood": 0, "Anxiety": 0, "Stress": 0}
        overall_correct = 0
        total_count = len(vignettes)

        cot_text = self.get_cot_text()

        for vignette_id, vignette_data in vignettes.items():
            category = vignette_data['category']
            vignette_text = vignette_data['vignette_text']
            ground_truth_label = vignette_data['label']

            rag_text = self.get_rag_text(vignette_text)
            model_diagnosis = self.generate_diagnosis(rag_text, vignette_text, cot_text).strip()
            final_diag = self.extract_diagnosis(model_diagnosis)

            score = self.compare_diagnosis(final_diag, ground_truth_label)
            if score >= 70:
                correct_counts[category] += 1
                overall_correct += 1
            total_counts[category] += 1

            print("------------------------------------------------------------------------------")
            print(f"Vignette ID: {vignette_id}")
            print(f"Category: {category}")
            print(f"Vignette Text:\n{vignette_text}\n")
            print(f"Model Diagnosis: {model_diagnosis}")
            print(f"The extracted Model Diagnosis: {final_diag}")
            print(f"Ground Truth Label: {ground_truth_label}")
            print("The similarity score provided by the model:", score)

        for category in correct_counts:
            category_accuracy = correct_counts[category] / total_counts[category] if total_counts[category] > 0 else 0
            print(f"Accuracy for {category}: {category_accuracy:.2f}")

        overall_accuracy = overall_correct / total_count if total_count > 0 else 0
        print(f"Overall Accuracy: {overall_accuracy:.2f}")
        
        Counts = {
            "Mood": correct_counts["Mood"] ,
            "Anxiety": correct_counts["Anxiety"],
            "Stress": correct_counts["Stress"],
            "Overall": overall_accuracy
        }
        Rates = {"Mood": correct_counts["Mood"] / total_counts["Mood"] if total_counts["Mood"] > 0 else 0,
            "Anxiety": correct_counts["Anxiety"] / total_counts["Anxiety"] if total_counts["Anxiety"] > 0 else 0,
            "Stress": correct_counts["Stress"] / total_counts["Stress"] if total_counts["Stress"] > 0 else 0,
            "Overall": overall_accuracy}
        return Counts, Rates


In [ ]:
def run_model(filepath, category, chunk_size, method):
    model = ModelHyperparameterTuning(chunk_size=chunk_size, method=method)
    vignettes = model.load_vignettes(filepath, category)
    #results = model.evaluate_model(vignettes)
    counts, rates = model.evaluate_model(vignettes)
    return rates
    
    
all_accuracies = []
for i in range(1):
    llama_model = Ollama(model="llama3.3")
    mistral_model = Ollama(model ='mistral')
    accuracy = run_model("Data_final.csv", "Stress", chunk_size=10, method="rag ")
    all_accuracies.append(accuracy)
    print(f"Run {i+1}: {accuracy}")
    gc.collect()
    
print("All Accuracies:", all_accuracies)


In [ ]:
all_accuracies